### This notebook is optionally accelerated with a GPU runtime.
### If you would like to use this acceleration, please select the menu option "Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and click "SAVE"

----------------------------------------------------------------------

# MiDaS

*Author: Intel ISL*

**MiDaS models for computing relative depth from a single image.**

<img src="https://pytorch.org/assets/images/midas_samples.png" alt="alt" width="50%"/>


### Model Description

[MiDaS](https://arxiv.org/abs/1907.01341) computes relative inverse depth from a single image. The repository provides multiple models that cover different use cases ranging from a small, high-speed model to a very large model that provide the highest accuracy. The models have been trained on 10 distinct datasets using
multi-objective optimization to ensure high quality on a wide range of inputs.

### Dependencies

MiDaS depends on [timm](https://github.com/rwightman/pytorch-image-models). Install with

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install h5py
import h5py

In [3]:
import scipy.io as sio
mat_path = "/content/drive/MyDrive/nyu_depth_v2_labeled.mat"

In [4]:
with h5py.File(mat_path, 'r') as f:
    print(list(f.keys()))
    # 这里会列出顶层数据集/组的名称，比如 "images", "depths", ...

    # 若想访问某个数据集，比如 'images'
    images = f['images'][:]  # 转成 numpy array
    depths = f['depths'][:]
    print(images.shape, depths.shape)

['#refs#', '#subsystem#', 'accelData', 'depths', 'images', 'instances', 'labels', 'names', 'namesToIds', 'rawDepthFilenames', 'rawDepths', 'rawRgbFilenames', 'sceneTypes', 'scenes']
(1449, 3, 640, 480) (1449, 640, 480)


In [7]:
import os
import h5py
import torch
import numpy as np
from torch.utils.data import Dataset

class NYUv2MatHDF5(Dataset):
    def __init__(self, mat_path, transform=None):
        """
        mat_path: 形如 "/content/drive/MyDrive/nyu_depth_v2_labeled.mat"
        transform: 可选，对 (image, depth) 做进一步缩放/增强
        """
        self.transform = transform

        # 读取 v7.3 (HDF5) 格式的 .mat 文件
        with h5py.File(mat_path, 'r') as f:
            # 这里要看具体有哪些键，通常 "images" / "depths" 是常见名称
            self.images = f['images'][:]   # => numpy array
            self.depths = f['depths'][:]   # => numpy array

        # 可能 shapes 是 (H, W, 3, N) / (H, W, N)
        # 取决于 nyu_depth_v2_labeled.mat 内部存储方式
        # 这里假设 images.shape = (H, W, 3, N), depths.shape = (H, W, N)
        # 转换到 (N, H, W, 3) / (N, H, W)
        self.images = self.images.transpose((0, 3, 2, 1))  # => (N,H,W,3)
        self.depths = self.depths.transpose((0, 2, 1))      # => (N,H,W)
        print("After transpose, self.images.shape =", self.images.shape)
        print("After transpose, self.depths.shape =", self.depths.shape)

        # 样本总数
        self.length = self.images.shape[0]

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        img = self.images[idx]   # shape (H,W,3)
        depth = self.depths[idx] # shape (H,W)

        # 转 float32
        img = img.astype(np.float32)
        depth = depth.astype(np.float32)

        # 若有自定义transform (e.g. resize/crop/augment)，这里执行
        if self.transform:
            img, depth = self.transform(img, depth)

        # 转 PyTorch Tensor:
        #   - 图像 [H,W,3] => [3,H,W]
        #   - 深度 [H,W] => [1,H,W]
        img_tensor = torch.from_numpy(img).permute(2,0,1)   # [3,H,W]
        depth_tensor = torch.from_numpy(depth).unsqueeze(0)  # [1,H,W]

        return img_tensor, depth_tensor


In [8]:
from torch.utils.data import DataLoader
mat_file = "/content/drive/MyDrive/nyu_depth_v2_labeled.mat"  # 你在Drive里的.mat文件路径
nyu_dataset = NYUv2MatHDF5(mat_file)

# 创建 DataLoader
nyu_loader = DataLoader(
    nyu_dataset,
    batch_size=4,   # 一次取4个样本
    shuffle=True,   # 打乱顺序
)

After transpose, self.images.shape = (1449, 480, 640, 3)
After transpose, self.depths.shape = (1449, 480, 640)


In [10]:
for batch_idx, (images, depths) in enumerate(nyu_loader):
    print("Batch index:", batch_idx)
    print("images shape:", images.shape)  # [B, 3, H, W]
    print("depths shape:", depths.shape)  # [B, 1, H, W]
    break

Batch index: 0
images shape: torch.Size([4, 3, 480, 640])
depths shape: torch.Size([4, 1, 480, 640])


In [ ]:
pip install timm

### Example Usage

Download an image from the PyTorch homepage

In [ ]:
import cv2
import torch
import urllib.request

import matplotlib.pyplot as plt

url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
urllib.request.urlretrieve(url, filename)

Load a model (see [https://github.com/intel-isl/MiDaS/#Accuracy](https://github.com/intel-isl/MiDaS/#Accuracy) for an overview)

In [ ]:
model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
#model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)

Move model to GPU if available

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

Load transforms to resize and normalize the image for large or small model

In [ ]:
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

Load image and apply transforms

In [ ]:
img = cv2.imread(filename)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

input_batch = transform(img).to(device)

Predict and resize to original resolution

In [ ]:
with torch.no_grad():
    prediction = midas(input_batch)

    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img.shape[:2],
        mode="bicubic",
        align_corners=False,
    ).squeeze()

output = prediction.cpu().numpy()

Show result

In [ ]:
plt.imshow(output)
# plt.show()

### References
[Towards Robust Monocular Depth Estimation: Mixing Datasets for Zero-shot Cross-dataset Transfer](https://arxiv.org/abs/1907.01341)

[Vision Transformers for Dense Prediction](https://arxiv.org/abs/2103.13413)

Please cite our papers if you use our models:

In [ ]:
@article{Ranftl2020,
	author    = {Ren\'{e} Ranftl and Katrin Lasinger and David Hafner and Konrad Schindler and Vladlen Koltun},
	title     = {Towards Robust Monocular Depth Estimation: Mixing Datasets for Zero-shot Cross-dataset Transfer},
	journal   = {IEEE Transactions on Pattern Analysis and Machine Intelligence (TPAMI)},
	year      = {2020},
}

In [ ]:
@article{Ranftl2021,
	author    = {Ren\'{e} Ranftl and Alexey Bochkovskiy and Vladlen Koltun},
	title     = {Vision Transformers for Dense Prediction},
	journal   = {ArXiv preprint},
	year      = {2021},
}